# Prevendo Atributo Protegido de Atributos não Protegidos

Ao conhecer a ideia de Fairness e dar os primeiros passos nessa área a primeira resolução mais óbvia dos problemas é:

Retirar o Atributo Protegido deve garantir a Igualdade, já que o Classificador vai deixar de classificar de acordo com esse Atributo, então não haverá mais problemas.

No entanto esse pequeno e rápido código é para demonstrar que apenas isso não resolve o problema.

Isso ocorre por conta da correlação que outros atributos tem com esse Atributo Protegido. Meu objetivo é, com essa demonstração rápida, enxergar essa relação e demonstrar como um classificador pode, facilmente prever esse atributo e que, devido a esse problema, devemos buscar alternativas mais viáveis e condizentes, para buscar uma solução que resolve o problema verdadeiramente.


In [130]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

data = pd.read_csv(".\\adult.csv")
data.info()
data = data.drop((data[data["native-country"] == "Holand-Netherlands"]).index)
# Fiz esse drop porque originalmente só tem 1 registro com esse native-country, ficava dando problema se ele não fosse pro treino então preferi retirá-lo

data1 = data.drop(data[(data['race'] != 'White') & (data['race'] != "Black")].index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


#### Há 3 formas de testar isso, sendo essas 3 formas MUITO parecidas:
#### Utilizando um classificador simples
Classificar em 2 classes, sendo elas "White" e "Black", ignorando os outras classes presentes;

Classificar em 2 classes, sendo elas "White" e "Other" referindo-se a todas as outras classes presentes;

Classificar em 4 classes, as raças já presentes no Dataset normalmente.

In [131]:
x1 = data1.drop(["income", "race"], axis = 1)
# Não vou utilizar "income" que é originalmente o atributo que desejamos prever, procuro mostrar a relação apenas entre os Atributos não Protegidos de X com o Atributo Protegido
Y1 = data1["race"]

x2 = data.drop(["income", "race"], axis = 1)
Y2 = data["race"]

from sklearn.model_selection import train_test_split

x_train1, x_test1, Y_train1, Y_test1 = train_test_split(x1, Y1, test_size = 0.3)
x_train2, x_test2, Y_train2, Y_test2 = train_test_split(x2, Y2, test_size = 0.3)

Para isso eu crio duas instâncias de x e Y, sendo a instância 1 os valores originais no entanto possuindo apenas as raças "White" e "Black" e a instâncias 2 todos os valores inicias, posteriormente a instância 2 vai virar outro Classificador já que a codificação para as classes vão ser diferentes.

In [132]:
from sklearn.preprocessing import OrdinalEncoder

categoricalLabels1 = (x_train1.select_dtypes(include = ['object'])).columns
categoricalLabels2 = (x_train2.select_dtypes(include = ['object'])).columns

encoder = OrdinalEncoder(dtype = 'int32')
encoder2 = OrdinalEncoder(dtype = 'int32')
encoder3 = OrdinalEncoder(dtype = 'int32')

num_x_train1 = x_train1.copy()
num_x_train2 = x_train2.copy()

num_x_test1 = x_test1.copy()
num_x_test2 = x_test2.copy()

Y_test3 = Y_test2.copy()
Y_train3 = Y_train2.copy()

num_x_train1[categoricalLabels1] = encoder.fit_transform(x_train1[categoricalLabels1])
num_x_test1[categoricalLabels1] = encoder.transform(x_test1[categoricalLabels1])

num_x_train2[categoricalLabels2] = encoder2.fit_transform(x_train2[categoricalLabels2])
num_x_test2[categoricalLabels2] = encoder2.transform(x_test2[categoricalLabels2])

num_Y_test1 = Y_test1.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 2, "Amer-Indian-Eskimo": 3, "Other": 4})
num_Y_train1 = Y_train1.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 2, "Amer-Indian-Eskimo": 3, "Other": 4})

num_Y_test2 = Y_test2.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 1, "Amer-Indian-Eskimo": 1, "Other": 1})
num_Y_train2 = Y_train2.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 1, "Amer-Indian-Eskimo": 1, "Other": 1})

num_Y_test3 = Y_test3.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 2, "Amer-Indian-Eskimo": 3, "Other": 4})
num_Y_train3 = Y_train3.replace({"White": 0, "Black": 1, "Asian-Pac-Islander": 2, "Amer-Indian-Eskimo": 3, "Other": 4})

# print()

## Classificador da Raça de uma Pessoa
Após fazer os tratamentos necessários, crio uma Árvore de Decisão para exibir os resultados 

In [133]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_absolute_error

modelo1 = DecisionTreeClassifier()
modelo2 = DecisionTreeClassifier()
modelo3 = DecisionTreeClassifier()

modelo1.fit(num_x_train1, num_Y_train1)
previsao1 = modelo1.predict(num_x_test1)

modelo2.fit(num_x_train2, num_Y_train2)
previsao2 = modelo2.predict(num_x_test2)

modelo3.fit(num_x_train2, num_Y_train3)
# Aqui eu posso utilizar o num_x_train2 e o num_x_test2 pois os testes 2 e 3 são exatamente iguais em x e só diferem em Y na forma da codificação das classes
# sendo a segunda forma apenas 2 classes 0 e 1, e a terceira 5 classes 0, 1, 2, 3 e 4
previsao3 = modelo3.predict(num_x_test2)

acuraciaNP = mean_absolute_error(num_Y_test1, previsao1)
print("Acurácia para a Forma 1(2 classes com exclusão de dados): ",acuraciaNP)
acuraciaNP = mean_absolute_error(num_Y_test2, previsao2)
print("Acurácia para a Forma 2(2 classes sem exclusão de dados): ",acuraciaNP)
acuraciaNP = mean_absolute_error(num_Y_test3, previsao3)
print("Acurácia para a Forma 3(5 classes sem exclusão de dados): ",acuraciaNP)

Acurácia para a Forma 1(2 classes com exclusão de dados):  0.1702310894215588
Acurácia para a Forma 2(2 classes sem exclusão de dados):  0.19975431652221387
Acurácia para a Forma 3(5 classes sem exclusão de dados):  0.3192520303009623


Os resultados dos 3 métodos foram:

1 - O erro é, em média, 0.16, variando pouco mais de 0.1 

2 - Erro em torno de 0.19 a 0.20

3 - Erro um pouco mais alto 0.31 variando entre 0.30 e 0.32, o que é compreensível dado ao número maior de classes e a pouca quantidade de dados para as classes minoritárias

Com isso, consigo enxergar que existe sim, uma correlação entre esses dados, já que, mesmo num classificador trivial, esse erro não foi tão alto nos 2 primeiros métodos, que nesse caso são mais razoáveis devido a quantidade de classes e quantidade de dados, assim é possível entender que retirar os Dados Sensíveis não é extremamente eficiente quanto parece. 

Em Classificadores melhor construídos, com técnicas mais avançadas esse erro poderia ser bem menor, mas como eu busco apenas uma demonstração rápida dessa relação é suficiente para mim esses resultados.